In [18]:
import pandas as pd
import json

In [ ]:
def clean_data(df):
    df["PROV"] = df["idep"] + df["iprov"]
    df["MUN"] = df["PROV"] + df["imun"]
    df.columns = df.columns.str.upper()
    df = df.rename(
        columns={
            "IDEP": "DEPAR",
            "I00": "N_VIV",
            "PROV": "PROV",
            "MUN": "MUN",
            "PAIS_DESTINO_COD": "PAIS_DES",
        }
    )
    df = df.drop(columns=["IMUN", "IPROV"])
    df = df[
        [
            "DEPAR",
            "PROV",
            "MUN",
            "N_VIV",
            "E203_SEXO",
            "E204_ANSAL",
            "E205_EDAD",
            "PAIS_DES",
        ]
    ]
    df = df.sort_values(by=["DEPAR", "PROV", "MUN", "N_VIV"])
    return df


df = pd.read_csv(r"Emigracion_CPV-2024.csv", dtype=str, sep=";", low_memory=False)

df_clean = clean_data(df.copy())
df_clean.to_csv("emigracion.csv", index=False)

In [22]:
fields_path = "../diccionario/fields.json"

In [23]:
with open(fields_path, "r", encoding="utf-8") as f:
    fields = json.load(f)

necessary_columns = df_clean.columns.tolist()
fields = [field for field in fields if field.get("name") in necessary_columns]

# Crear el datapackage
datapackage = {
    "name": "emigracion",
    "resources": [
        {
            "name": "df_clean",
            "path": "emigracion.csv",
            "schema": {"fields": fields}
        }
    ]
}

# Guardar el datapackage en un archivo JSON
with open("datapackage.json", "w", encoding="utf-8") as f:
    json.dump(datapackage, f, ensure_ascii=False, indent=4)